* https://uniswapv3book.com/index.html
* https://medium.com/@chaisomsri96/defi-math-uniswap-v3-concentrated-liquidity-bd87686b3ecf
* https://liobaheimbach.github.io/assets/pdf/Papers/Risks_and_Returns_of_Uniswap_V3_Liquidity_Providers.pdf
* https://docs.uniswap.org/sdk/v3/guides/advanced/active-liquidity
* https://blog.uniswap.org/uniswap-v3-math-primer
* https://blog.uniswap.org/uniswap-v3-math-primer-2
* https://atiselsts.github.io/pdfs/uniswap-v3-liquidity-math.pdf

In [1]:
from uniswappy import *
import pandas as pd
import copy
import matplotlib.pyplot as plt

user_nm = MockAddress().apply()
eth_amount = 10000
tkn_amount = 100000

fee = UniV3Utils.FeeAmount.MEDIUM
tick_spacing = UniV3Utils.TICK_SPACINGS[fee]
lwr_tick = UniV3Utils.getMinTick(tick_spacing)
upr_tick = UniV3Utils.getMaxTick(tick_spacing)

In [2]:
def calc_Lx(sqrtP, dx, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = sqrtP
    Lx = dx/(1/p_sqrt_human - 1/pb_sqrt_human)
    return Lx

def calc_Ly(sqrtP, dy, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = sqrtP
    Ly = dy/(p_sqrt_human - pa_sqrt_human)
    return Ly

def calc_y(dL, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = lp.slot0.sqrtPriceX96/2**96
    dy = dL*(p_sqrt_human - pa_sqrt_human)
    #y = dL*(pb_sqrt_human - pa_sqrt_human)
    return dy

def calc_x(dL, lwr_tick, upr_tick):
    pa_sqrt_human = TickMath.getSqrtRatioAtTick(lwr_tick)/2**96
    pb_sqrt_human = TickMath.getSqrtRatioAtTick(upr_tick)/2**96 
    p_sqrt_human = lp.slot0.sqrtPriceX96/2**96
    dx = dL*(1/p_sqrt_human - 1/pb_sqrt_human)
    #x = dL*(1/p_sqrt_human - 1/pb_sqrt_human)
    return dx

In [3]:
def get_reserves(lp, token_in):
    tokens = lp.factory.token_from_exchange[lp.name]
    if(lp.version == UniswapExchangeData.VERSION_V2):
        if(token_in.token_name == lp.token1):
            x = lp.get_reserve(tokens[lp.token0])
            y = lp.get_reserve(tokens[lp.token1])
        else: 
            x = lp.get_reserve(tokens[lp.token1])
            y = lp.get_reserve(tokens[lp.token0])
    elif(lp.version == UniswapExchangeData.VERSION_V3):   
        if(token_in.token_name == lp.token0):
            x = lp.get_reserve(tokens[lp.token0])
            y = lp.get_reserve(tokens[lp.token1])
        else: 
            x = lp.get_reserve(tokens[lp.token1])
            y = lp.get_reserve(tokens[lp.token0]) 
    return (x, y)  

def calc_lp_settlement(lp, token_in, itkn_amt, lwr_tick, upr_tick):

    L = lp.get_liquidity()
    #(x, y) = get_reserves(lp, token_in) 
    x = calc_y(L, lwr_tick, upr_tick)
    y = calc_x(L, lwr_tick, upr_tick)
    #print(f'x {x}, y {y}')
    
    gamma = 997

    a1 = x*y/L
    a2 = L
    a = a1/a2
    b = (1000*itkn_amt*x - itkn_amt*gamma*x + 1000*x*y + x*y*gamma)/(1000*L);
    c = itkn_amt*x;

    dL = (b*a2 - a2*math.sqrt(b*b - 4*a1*c/a2)) / (2*a1);
    return dL

def calc_withdraw_portion(lp, token_in, amt, lwr_tick, upr_tick):

    L = lp.get_liquidity()
    #(x, y) = get_reserves(lp, token_in)
    x = calc_y(L, lwr_tick, upr_tick)
    y = calc_x(L, lwr_tick, upr_tick)    
    gamma = 997/1000

    dL = calc_lp_settlement(lp, token_in, amt, lwr_tick, upr_tick)
    dx = dL*x/L
    dy = dL*y/L
    aswap = (gamma*dx)*(y-dy)/(x-dx+gamma*dx)

    return dy/amt 

def calc_lp_settlement2(lp, token_in, q, lwr_tick, upr_tick):

    L = lp.get_liquidity()
    dPy = calc_y(1, lwr_tick, upr_tick)
    dPx = calc_x(1, lwr_tick, upr_tick)
    (x, y) = get_reserves(lp, token_in)
    
    a = dPx*dPy
    b = (1000*q*dPx - 997*q*dPx + 1000*dPy*x + 997*y*dPx)/(1000);
    c = q*x;

    dL = (b - math.sqrt(b*b - 4*a*c)) / (2*a);
    return dL


In [4]:
q96 = 2**96
def price_to_sqrtp(p):
    return int(math.sqrt(p) * q96)

def tick_to_price(tick):
    return math.floor(1.0001**tick)

def price_to_tick(p):
    return math.floor(math.log(p, 1.0001))

def liquidity0(amount, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return (amount * (pa * pb) / q96) / (pb - pa)

def liquidity1(amount, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return amount * q96 / (pb - pa)

def calc_amount0(liq, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return int(liq * q96 * (pb - pa) / pa / pb)


def calc_amount1(liq, pa, pb):
    if pa > pb:
        pa, pb = pb, pa
    return int(liq * (pb - pa) / q96)

def price_to_tick(p):
    return math.floor(math.log(p, 1.0001))

In [5]:
import math

eth = ERC20("ETH", "0x09")
tkn = ERC20("TKN", "0x111")
init_price = UniV3Utils.encodePriceSqrt(10, 1)

exchg_data = UniswapExchangeData(tkn0 = eth, tkn1 = tkn, symbol="LP", 
                                   address="0x011", version = 'V3', 
                                   tick_spacing = tick_spacing, 
                                   fee = fee)

factory = UniswapFactory("ETH pool factory", "0x2")
lp = factory.deploy(exchg_data)

lwr_tick = lp.get_tick_price(-1, 10, 1000)
upr_tick = lp.get_tick_price(1, 10, 1000)

#lwr_tick = UniV3Utils.getMinTick(tick_spacing)
#upr_tick = UniV3Utils.getMaxTick(tick_spacing)

sqrtP =  math.sqrt(10)
Ly = calc_Ly(sqrtP, 100, lwr_tick, upr_tick)
Lx = calc_Lx(sqrtP, 1000, lwr_tick, upr_tick)

lp.initialize(init_price)
out = lp.mint(user_nm, lwr_tick, upr_tick, Lx)    #3162.2776601683795
lp.summary()

lwr_tick = lp.get_tick_price(-1, lp.get_price(eth), 1000)
upr_tick = lp.get_tick_price(1, lp.get_price(eth), 1000)
#lwr_tick = UniV3Utils.getMinTick(tick_spacing)
#upr_tick = UniV3Utils.getMaxTick(tick_spacing)

amount_out = 100
Lx = calc_Lx(sqrtP, amount_out, lwr_tick, upr_tick)
p_out = calc_withdraw_portion(lp, tkn, amount_out, lwr_tick, upr_tick)
#dL2 = calc_lp_settlement(lp, eth, amount_out, lwr_tick2, upr_tick2)
#dL = calc_lp_settlement2(lp, eth, amount_out, lwr_tick2, upr_tick2)
#print(f'dL {dL2} dL2 {dL}')

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 999.9999999999997, TKN = 10340.759310919504
Gross Liquidity: 66597.92378128914 



In [6]:
dL = 3283.3309207565794

# Burn portion
dPy = calc_y(1, lwr_tick, upr_tick)
dPx = calc_x(1, lwr_tick, upr_tick)
dx = dL*dPx
dy = dL*dPy

# Swap portion
eth_gwei = 10**18
L = lp.get_liquidity()
liq = (L - dL) * eth_gwei
amount_in = dy * eth_gwei
sqrtp_cur = price_to_sqrtp(lp.get_price(eth))

price_diff = ((amount_in - 3*amount_in/1000) * q96) // liq
price_next_calc = sqrtp_cur + price_diff
#price_next = math.sqrt(10.05083685047008)*q96
#print("New price (actual):", (price_next / q96) ** 2)
print("New price (calc):", (price_next_calc / q96) ** 2)
print("New sqrtP:", price_next_calc)
print("New tick:", price_to_tick((price_next_calc / q96) ** 2))

amount_in = calc_amount1(liq, price_next_calc, sqrtp_cur)
amount_out = calc_amount0(liq, price_next_calc, sqrtp_cur)

print("USDC in:", amount_in / eth_gwei)
print("ETH out:", amount_out / eth_gwei)

dx_swap = amount_out / eth_gwei

# x pulled out
dx + dx_swap

New price (calc): 10.05083685047008
New sqrtP: 2.5117747796363857e+29
New tick: 23077
USDC in: 508.2782835158756
ETH out: 50.69912253362368


99.99992253362367

In [7]:
p_out = 0.493008
(_, _, _, _, amount0, amount1) = lp.burn(user_nm, lwr_tick, upr_tick, p_out*Lx)
lp.summary()

out = Swap().apply(lp, tkn, user_nm, amount1) 
lp.summary()
print(f'amount0 {amount0}, amount1 {amount1}')
out[1]

Exchange ETH-TKN (LP)
Real Reserves:   ETH = 950.6991999999997, TKN = 9830.951604283724
Gross Liquidity: 63314.59286053256 

new price 251177477963638562769788518347
Exchange ETH-TKN (LP)
Real Reserves:   ETH = 900.0000774663762, TKN = 10340.759310919504
Gross Liquidity: 63314.59286053256 

amount0 49.30079999999998, amount1 509.8077066357803


-50.69912253362341